In [1]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.8 MB/s eta 0:00:00


In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import os
import pickle
from PIL import Image
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from tensorflow.keras.models import load_model

# Constants
FEMALE_GENDER = 1
MALE_GENDER = 2
IMG_SIZE = (224, 224)

def main():
    # Load the trained models
    rf_model = None
    try:
        with open('random_forest_model.pkl', 'rb') as file:
            rf_model = pickle.load(file)
    except Exception as e:
        st.error(f"Failed to load the RandomForestClassifier model: {e}")

    cnn_model = None
    try:
        cnn_model = load_model('your_cnn_model.h5')
    except Exception as e:
        st.error(f"Failed to load the CNN model: {e}")

    # Load the dataset
    dataset = pd.read_csv('dataset.csv')

    # Streamlit app
    st.title("Body Measurement Classifier")

    # Gender selection
    gender = st.radio("Gender:", ["Female", "Male"])

    # Age input
    age = st.number_input("Age:", step=1, value=0)

    # Measurement inputs
    measurement_labels = ["Shoulder", "Waist", "Hips", "Bust", "Chest"]
    measurements = []
    for label in measurement_labels:
        measurement = st.number_input(label + ":", value=0.0)
        measurements.append(measurement)

    # Classify button
    if st.button("Classify"):
        try:
            # Create DataFrame from user inputs
            data = pd.DataFrame(columns=['Gender', 'Age', 'Shoulder', 'Waist', 'Hips', 'Bust', 'Chest'])
            data.loc[0] = [FEMALE_GENDER if gender == "Female" else MALE_GENDER, age] + measurements

            # Predict body type using RandomForestClassifier
            if rf_model:
                body_type = rf_model.predict(data)
                st.success(f"Predicted Body Type: {body_type[0]}")

                # Provide recommendations based on predicted body type
                provide_recommendations(body_type[0], gender)
            else:
                st.error("RandomForestClassifier Model not loaded.")
        except Exception as e:
            st.error(str(e))

def provide_recommendations(body_type, gender):
    try:
        # Filter dataset based on predicted body type and gender
        filtered_data = dataset[(dataset['Gender'] == gender) & (dataset[body_type[0]] == 1)]

        # Retrieve top 5 images for each cloth pattern
        recommendations = {}
        for cloth_pattern in filtered_data['Cloth Pattern '].unique():
            pattern_data = filtered_data[filtered_data['Cloth Pattern '] == cloth_pattern]
            top_images = pattern_data.head(5)['Image Path'].tolist()
            recommendations[cloth_pattern] = top_images

        # Display recommendations
        display_recommendations(recommendations)
    except Exception as e:
        st.error(str(e))

def display_recommendations(recommendations):
    for cloth_pattern, images in recommendations.items():
        st.subheader(f"Top 5 images for {cloth_pattern}:")
        cols = st.columns(5)
        for i, image_path in enumerate(images):
            full_image_path = os.path.join(os.getcwd(), image_path)
            img = Image.open(full_image_path)
            cols[i].image(img, caption=f"Image {i+1}", use_column_width=True)

# Run the Streamlit app
if __name__ == "__main__":
    st.set_page_config(layout="wide")  # Set the page layout to wide
    main()